<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/main/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [ ]:
colab = False

In [ ]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt
    !git checkout seq2seq

# 1.0 Preparation


In [ ]:
N_TRAIN = 50000
N_VAL = 9842
N_TEST = 9824

## 1.1 Loading Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("esnli", download_mode="force_redownload")

In [ ]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

In [ ]:
training_set[0]

In [ ]:
train_small = training_set.select(range(N_TRAIN))
valid_small = validation_set.select(range(N_VAL))
test_small = test_set.select(range(N_TEST))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

## 1.2 Loading T5 Model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-base", truncation=True, padding=True)
model = T5ForConditionalGeneration.from_pretrained("t5-base")

Test **zero-shot** on a random task.

In [ ]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_length=20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=20))

## 1.3 Zero-shot example to Verify Everything is Working

In [ ]:
from src.utils import generate_prompt_mnli

In [ ]:
example = training_set[0]
example

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [ ]:
prompt = generate_prompt_mnli(example)
prompt

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(outputs)
print("Shape of outputs:", outputs.shape)
print("Shape of outputs[0]:", outputs[0].shape)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 1.4  Tokenize the dataset

In [ ]:
train_small.info

In [ ]:
train_small.features

In [ ]:
from functools import partial
from src.utils import tokenize_function

In [ ]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [ ]:
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)

print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

In [ ]:
train_small_tokenized = train_small_tokenized.remove_columns(["label"])
valid_small_tokenized = valid_small_tokenized.remove_columns(["label"])
test_small_tokenized = test_small_tokenized.remove_columns(["label"])

In [ ]:
train_small_tokenized.features

# 2.0 Tasks

### Imports and definitions

In [ ]:
import torch
from functools import partial
import evaluate
from src.utils import compute_metrics, eval_pred_transform_accuracy
from transformers import (
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

In [ ]:
transform_accuracy = partial(eval_pred_transform_accuracy, tokenizer=tokenizer)
compute_accuracy = partial(
    compute_metrics, pred_transform=transform_accuracy, metric=evaluate.load("accuracy")
)

## 2.1 Task 1: Zero-shot evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="task1",
    predict_with_generate=True,
    per_device_eval_batch_size=16,
    generation_max_length=32,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(test_small_tokenized)

## 2.2 Task 2: Fine tuning without explanations

In [ ]:
NUM_EPOCHS = 5

In [ ]:
model_ft = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator_ft = DataCollatorForSeq2Seq(tokenizer, model=model_ft)

In [ ]:
training_args_ft = Seq2SeqTrainingArguments(
    save_strategy="no",
    output_dir="task2",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=32,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft = Seq2SeqTrainer(
    model=model_ft,
    args=training_args_ft,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    data_collator=data_collator_ft,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft.train()

In [ ]:
trainer_ft.evaluate(test_small_tokenized)

## 2.3 Task 3: Fine Tuning with Explanations

We need to give as labels the label and the explanation tokenized.

### Preparing the dataset with labelled explanations

In [ ]:
dataset_explanations = load_dataset("esnli", download_mode="force_redownload")

In [ ]:
training_set_ex = dataset_explanations["train"]
validation_set_ex = dataset_explanations["validation"]
test_set_ex = dataset_explanations["test"]

print("Shape of training_set: ", training_set_ex.shape)
print("Shae of validation_set: ", validation_set_ex.shape)
print("Shape of test_set: ", test_set_ex.shape)

In [ ]:
train_small_ex = training_set_ex.select(range(N_TRAIN))
valid_small_ex = validation_set_ex.select(range(N_VAL))
test_small_ex = test_set_ex.select(range(N_TEST))

print("Shape of train_small: ", train_small_ex.shape)
print("Shape of valid_small: ", valid_small_ex.shape)
print("Shape of test_small: ", test_small_ex.shape)

#### Tokenizing the dataset

In [ ]:
from functools import partial
from src.utils import tokenize_function_ex

In [ ]:
tokenize_mapping_ex = partial(tokenize_function_ex, tokenizer=tokenizer)

In [ ]:
train_small_tokenized_ex = train_small_ex.map(
    tokenize_mapping_ex, batched=True
).with_format("torch")
valid_small_tokenized_ex = valid_small_ex.map(
    tokenize_mapping_ex, batched=True
).with_format("torch")
test_small_tokenized_ex = test_small_ex.map(
    tokenize_mapping_ex, batched=True
).with_format("torch")

print("Shape of train_small_tokenized: ", train_small_tokenized_ex.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized_ex.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized_ex.shape)

In [ ]:
train_small_tokenized_ex = train_small_tokenized_ex.remove_columns(["label"])
valid_small_tokenized_ex = valid_small_tokenized_ex.remove_columns(["label"])
test_small_tokenized_ex = test_small_tokenized_ex.remove_columns(["label"])

In [ ]:
train_small_tokenized_ex.features

### Fine Tuning

In [ ]:
NUM_EPOCHS = 5

In [ ]:
transform_accuracy_ex = partial(
    eval_pred_transform_accuracy,
    tokenizer=tokenizer,
    remove_explanations_from_label=True,
)
compute_accuracy_removing_explanations = partial(
    compute_metrics,
    pred_transform=transform_accuracy_ex,
    metric=evaluate.load("accuracy"),
)

In [ ]:
model_ft_ex = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator_ft_ex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [ ]:
training_args_ft_ex = Seq2SeqTrainingArguments(
    save_strategy="no",
    output_dir="task3",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft_ex = Seq2SeqTrainer(
    model=model_ft_ex,
    args=training_args_ft_ex,
    train_dataset=train_small_tokenized_ex,
    eval_dataset=valid_small_tokenized_ex,
    compute_metrics=compute_accuracy_removing_explanations,
    data_collator=data_collator_ft_ex,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft_ex.train()

In [ ]:
trainer_ft_ex.evaluate(test_small_tokenized_ex)

## 2.4 Task 4: Fine Tuning with Shuffled Explanations

### Preparing the dataset with *wrong* labelled explanations

In [ ]:
texts = []
for example in train_small:  
    texts.append(example["explanation_1"])

# Save the texts to a text file
with open("ex_files/explanations_train.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")

In [ ]:
texts = []
for example in valid_small: 
    texts.append(example["explanation_1"])

# Save the texts to a text file
with open("ex_files/explanations_val.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")

In [ ]:
texts = []
for example in test_small:  
    texts.append(example["explanation_1"])

# Save the texts to a text file
with open("ex_files/explanations_test.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")

In [ ]:
import random
input_file = "ex_files/explanations_train.txt"
output_file = "ex_files/shuffled_explanations_train.txt"

with open(input_file, "r") as f:
    lines = f.readlines()

random.shuffle(lines)

with open(output_file, "w") as f:
    f.writelines(lines)

In [ ]:
input_file = "ex_files/explanations_val.txt"
output_file = "ex_files/shuffled_explanations_val.txt"

with open(input_file, "r") as f:
    lines = f.readlines()

random.shuffle(lines)

with open(output_file, "w") as f:
    f.writelines(lines)

In [ ]:
input_file = "ex_files/explanations_test.txt"
output_file = "ex_files/shuffled_explanations_test.txt"

with open(input_file, "r") as f:
    lines = f.readlines()

random.shuffle(lines)

with open(output_file, "w") as f:
    f.writelines(lines)

In [ ]:
with open("ex_files/shuffled_explanations_train.txt", "r") as f:
    shuffled_explanations_train = f.readlines()

with open("ex_files/shuffled_explanations_train.txt", "r") as f:
    shuffled_explanations_val = f.readlines()

with open("ex_files/shuffled_explanations_test.txt", "r") as f:
    shuffled_explanations_test = f.readlines()

In [ ]:
from src.utils import tokenize_function_ex

tokenize_mapping_train = partial(
    tokenize_function_ex, tokenizer=tokenizer, modified_explanations = shuffled_explanations_train
)

tokenize_mapping_val = partial(
    tokenize_function_ex, tokenizer=tokenizer, modified_explanations = shuffled_explanations_val
)

tokenize_mapping_test = partial(
    tokenize_function_ex, tokenizer=tokenizer, modified_explanations = shuffled_explanations_test
)

In [ ]:
train_small_tokenized_shex = train_small.map(
    tokenize_mapping_train, batched=True
).with_format("torch")

valid_small_tokenized_shex = valid_small.map(
    tokenize_mapping_val, batched=True
).with_format("torch")

test_small_tokenized_shex = test_small.map(tokenize_mapping_test, batched=True).with_format(
    "torch"
)

print("Shape of train_small_tokenized: ", train_small_tokenized_shex.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized_shex.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized_shex.shape)

In [ ]:
train_small_tokenized_shex = train_small_tokenized_shex.remove_columns(["label", "explanation_1", "explanation_2", "explanation_3"])
valid_small_tokenized_shex = valid_small_tokenized_shex.remove_columns(["label", "explanation_1", "explanation_2", "explanation_3"])
test_small_tokenized_shex = test_small_tokenized_shex.remove_columns(["label", "explanation_1", "explanation_2", "explanation_3"])

### Fine Tuning

In [ ]:
NUM_EPOCHS = 5

In [ ]:
transform_accuracy_shex = partial(
    eval_pred_transform_accuracy,
    tokenizer=tokenizer,
    remove_explanations_from_label=True,
)
compute_accuracy_removing_explanations_shex = partial(
    compute_metrics,
    pred_transform=transform_accuracy_ex,
    metric=evaluate.load("accuracy"),
)

In [ ]:
model_ft_shex = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator_ft_shex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [ ]:
training_args_ft_shex = Seq2SeqTrainingArguments(
    save_strategy="no",
    output_dir="task4",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft_shex = Seq2SeqTrainer(
    model=model_ft_shex,
    args=training_args_ft_shex,
    train_dataset=train_small_tokenized_shex,
    eval_dataset=valid_small_tokenized_shex,
    compute_metrics=compute_accuracy_removing_explanations_shex,
    data_collator=data_collator_ft_shex,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft_shex.train()

In [ ]:
trainer_ft_shex.evaluate(test_small_tokenized_shex)

## 2.5 Task 5: Profiling-UD

### Read the results of the automatic annotation stage performed over explanations with Profilind-UD.

1. **Token ID**: The token's position in the sentence.
2. **Token**: The actual token text.
3. **Lemma**: The lemma or base form of the token.
4. Universal part-of-speech tag.
5. Language-specific part-of-speech tag (optional).
6. Miscellaneous (misc) field, which can contain additional annotations.
7. Head: The ID of the token's syntactic head.
8. Dependency relation: The type of syntactic relation between the token and its head.
9. Secondary dependencies or additional annotations.

In [ ]:
import pandas as pd 
# Define the path to your CoNLL-U file
conll_file_path = "ex_files/explanations_train.conllu"

# Define column names for the CoNLL-U file
column_names = [
    "ID",
    "TOKEN",
    "LEMMA",
    "UPOS",
    "XPOS",
    "FEATS",
    "HEAD",
    "DEPREL",
    "DEPS",
    "MISC"
]

# Read the CoNLL-U file into a DataFrame
df = pd.read_csv(conll_file_path, delimiter='\t', comment='#', header=None, names=column_names)

# Reset the index to create a numeric index
df.reset_index(drop=True, inplace=True)

# Display the DataFrame
df[:15]

In [ ]:
df['SAMPLE'] = None

sample = 0
for index, row in df.iterrows():
    if(row["ID"]==1):
        sample = sample+1
    df.at[index, "SAMPLE"] = sample

### Prepare the dataset with modified explanations

### Prepare the dataset with modified explanations

In [ ]:
# Define the input and output file paths
output_file = "ex_files/modified_explanations_1.txt"

# Write the shuffled lines to the output file
with open(output_file, "w") as f:
    for i in range(N_TRAIN):
        df_i = df.loc[df["SAMPLE"]==i]
        modified_exp = ' '.join(df["LEMMA"].values)
        f.writelines(modified_exp)

In [ ]:
with open("ex_files/modified_explanations_1.txt", "r") as f:
    explanations_m1 = f.readlines()

In [ ]:
# Define the input and output file paths
output_file = "ex_files/modified_explanations_1.txt"

# Write the shuffled lines to the output file
with open(output_file, "w") as f:
    for i in range(N_TRAIN):
        df_i = df.loc[df["SAMPLE"]==i]
        modified_exp = ' '.join(df["LEMMA"].values)
        f.writelines(modified_exp)

In [ ]:
with open("ex_files/modified_explanations_1.txt", "r") as f:
    explanations_m1 = f.readlines()